In [6]:
def sparse_eigs_vec(ham, n_eigs, n_vec_lanczos, sigma=0):
        """Compute eigenenergies using MUMPS as a sparse solver.

        Parameters:
        ----------
        ham : coo_matrix
            The Hamiltonian of the system in sparse representation..
        n_eigs : int
            The number of energy eigenvalues to be returned.
        n_vec_lanczos : int
            Number of Lanczos vectors used by the sparse solver.
        sigma : float
            Parameter used by the shift-inverted method. See
            documentation of scipy.sparse.linalg.eig

        Returns:
        --------
        A list containing the sorted energy levels. Only positive
        energies are returned.
        """
        class LuInv(sla.LinearOperator):
            def __init__(self, A):
                inst = kwant.linalg.mumps.MUMPSContext()
                inst.factor(A, ordering='metis')
                self.solve = inst.solve
                try:
                    super(LuInv, self).__init__(shape=A.shape, dtype=A.dtype,
                                                matvec=self._matvec)
                except TypeError:
                    super(LuInv, self).__init__(shape=A.shape, dtype=A.dtype)

            def _matvec(self, x):
                return self.solve(x.astype(self.dtype))

        ev, evecs = sla.eigs(ham, k=n_eigs,
                             OPinv=LuInv(ham), sigma=sigma, ncv=n_vec_lanczos)

        return ev, evecs